In [2]:
import numpy as np
import matplotlib.pylab as plt
import cv2 as cv

from ipywidgets import interact, interactive, fixed, interact_manual, SelectionSlider, FloatSlider
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

%matplotlib inline
from raytracing_lib import *
plt.style.use('seaborn-whitegrid')

In [3]:
# sellmeier refractive coefficients
materials = dict()
materials['BK7'] = [1.03961212, 0.00600069867, 0.231792344, 0.0200179144, 1.01046945, 103.560653]
materials['SSK2'] = [1.4306027, 0.00823982975, 0.153150554, 0.0333736841, 1.01390904, 106.870822]
materials['SF57'] = [1.81651371, 0.0143704198, 0.428893641, 0.0592801172, 1.07186278, 121.419942]

In [4]:
def plot_refractive_lenses(pos_ls_x, pos_ls_y, n_rays, alpha, sensor_position, sensor_height, sensor_size):
    global sensor_pos_global
    sensor_pos_global = sensor_position
    # prepare plots  
    fig = plt.figure(figsize=(30, 10), dpi=90)
    gs = gridspec.GridSpec(ncols=5, nrows=3, figure=fig)
    ax = fig.add_subplot(gs[0:3,0:4]) 
    ax2 = fig.add_subplot(gs[1,4]) 
    ax.set_facecolor((90/255, 90/255, 90/255))
    ax.grid(color=(0/255,0/255,0/255), linewidth=1, alpha=0.1)
    
    # find the max lens height
    global lenses
    max_lens_height = 0
    for lens in lenses:
        if lens.height > max_lens_height:
            max_lens_height = lens.height

    # set figure limits accordingly
    ax.set_xlim(0,(sensor_position + 1))
    ax.set_ylim(-(max_lens_height + 2),(max_lens_height + 2))
    ax.set_aspect(1)

    #plot sensor
    ax.plot([sensor_position, sensor_position],[sensor_height-sensor_size, sensor_height+sensor_size],'k', linewidth=3)

    # draw lenses
    for lens in lenses:
        lens.plot(ax)

    # calculate angle gamma between rays
    beta = np.arctan2(pos_ls_y, lenses[0].start_pos - pos_ls_x)
    if n_rays > 1:
        gamma = (2 * alpha)/(n_rays - 1)
        one_ray_angle = 0
    else:
        gamma = 0
        one_ray_angle = alpha

    # create container for all rays
    global rays
    rays = []     
    rays.clear() 

    # generate ray objects
    for wl in wavelengths:
        for i in range(int(n_rays)):
            angle = alpha - i * gamma - beta - one_ray_angle         
            rays.append(C_ray(pos_ls_x, pos_ls_y, wl, n_environment, angle=angle)) 

    # raytracing
    rays, intersections = raytracing(sensor_position, lenses, rays, n_environment, continue_ray_whithout_intersection=False)

    # plot rays and intersections
    for ray in rays:
            ax.plot(ray.x, ray.y, color=wavelength_to_rgb(ray.wavelength*1e9), linewidth=1)
    # for Intersection in intersections:
    #         ax.plot(Intersection[0], Intersection[1], Intersection[2], color=Intersection[3])

    # plot light source
    ax.plot(pos_ls_x,pos_ls_y, 'o', color=(255/255,200/255,0/255))   


    #plot sensor
    if len(wavelengths) == 3:
        sensor_color1_data= []
        sensor_color2_data= []
        sensor_color3_data= []
        for ray in rays:
            if np.isclose(ray.x[-1], sensor_pos_global):
                if ray.wavelength == wavelengths[0]:
                    sensor_color1_data.append(ray.y[-1])
                elif ray.wavelength == wavelengths[1]:
                    sensor_color2_data.append(ray.y[-1])
                elif ray.wavelength == wavelengths[2]:
                    sensor_color3_data.append(ray.y[-1])
                else:
                    print("ERROR while plotting sensor")
        sns.set_style('whitegrid')
        bw_method = 0.175
        sns.kdeplot(sensor_color1_data, bw_method=bw_method, ax=ax2, color=wavelength_to_rgb(wavelengths[0]*1e9))
        sns.kdeplot(sensor_color2_data, bw_method=bw_method, ax=ax2, color=wavelength_to_rgb(wavelengths[1]*1e9))
        sns.kdeplot(sensor_color3_data, bw_method=bw_method, ax=ax2, color=wavelength_to_rgb(wavelengths[2]*1e9))
        ax2.set_facecolor((90/255, 90/255, 90/255))
        ax2.grid(color=(0/255,0/255,0/255), linewidth=1, alpha=0.1)
        ax2.set_xlim(sensor_height-sensor_size, sensor_height+sensor_size)
    else:
        sensor_x_data = []
        sensor_x_data.clear()
        for ray in rays:
            if np.isclose(ray.x[-1], sensor_pos_global):
                sensor_x_data.append(ray.y[-1])
        sns.set_style('whitegrid')
        sns.kdeplot(sensor_x_data, bw_method=0.1, ax=ax2, color = 'orange')
        ax2.set_facecolor((90/255, 90/255, 90/255))
        ax2.grid(color=(0/255,0/255,0/255), linewidth=1, alpha=0.1)
        ax2.set_xlim(sensor_height-sensor_size, sensor_height+sensor_size)

    plt.show()

In [5]:
radius = 5
wavelengths = np.linspace(450e-9,650e-9,3)

n_environment = 1 # air

mat1 = 'SSK2'
mat2 = 'BK7'
mat3 = 'SF57'

pos_focus_lens_group = 0
max_lens_height = 6

# lens groups
LG1 = C_lens_group(5, 11, 30, 1.5, materials[mat1], max_lens_height)
LG1.add_lens(0.3, 33, 8, 0.5, materials[mat2], 0.9*max_lens_height)

LG2 = C_lens_group(LG1.group_end_pos + 3.4 + pos_focus_lens_group, -13, -5.5, 2, materials[mat1], 0.78*max_lens_height)
LG2.add_lens(0, -5.5, 10, 0.3, materials[mat2], 0.78*max_lens_height)
LG2.add_lens(0, 10, -16, 2, materials[mat3], 0.78*max_lens_height)

LG3 = C_lens_group(LG2.group_end_pos + 2 - pos_focus_lens_group, 13, 23, 0.6, materials[mat1], 0.7*max_lens_height)
LG3.add_lens(0, 11, -14, 1.5, materials[mat1], 0.66*max_lens_height)
LG3.add_lens(0, -14, 9, 0.15, materials[mat3], 0.66*max_lens_height)

LG4 = C_lens_group(LG3.group_end_pos + 2.2, -7, 13, 0.15, materials[mat3], 0.55*max_lens_height)
LG4.add_lens(0, 13, -11, 1.2, materials[mat1], 0.55*max_lens_height)
LG4.add_lens(0, -30, -8, 0.8, materials[mat1], 0.55*max_lens_height)
LG4.add_lens(0, -8, -50, 0.15, materials[mat3], 0.55*max_lens_height)
LG4.add_lens(0, 20, -8, 1.3, materials[mat1], 0.55*max_lens_height)

#concatonate all lens groups
lenses = LG1.lenses + LG2.lenses + LG3.lenses + LG4.lenses
sensor_position = lenses[-1].start_pos +lenses[-1].d + 16.8
 
inter = interact(plot_refractive_lenses,
    pos_ls_x=FloatSlider(value = -155, min=-300, max=lenses[0].start_pos-0.5, step=0.01),
    pos_ls_y=FloatSlider(value = 0, min=-2*radius, max=2*radius, step=0.001),
    n_rays=FloatSlider(value = 31, min=2, max=150, step=1),
    alpha=FloatSlider(value = 0.02, min=0.01, max=0.04, step=0.001),
    sensor_position=FloatSlider(value=sensor_position, min=sensor_position-3, max=sensor_position+10, step=0.1),
    sensor_height=FloatSlider(value=0, min=-1.35, max=1.35, step=0.01),
    sensor_size=FloatSlider(value=0.25, min=0.1, max=2, step=0.05))

interactive(children=(FloatSlider(value=-155.0, description='pos_ls_x', max=4.5, min=-300.0, step=0.01), Float…